In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score, cohen_kappa_score, precision_score, recall_score, accuracy_score, confusion_matrix
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from keras.layers.normalization import BatchNormalization
from keras.models import Model, Sequential
from keras.applications.xception import Xception
from keras.applications import *
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from PIL import Image
import random
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier 
from keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.decomposition import PCA

In [5]:
base_dir2 = '/Users/sandy/Desktop/Bigger Dataset'
train_dir = os.path.join(base_dir2, 'train')
validation_dir = os.path.join(base_dir2, 'validation')
test_dir = os.path.join(base_dir2, 'test')

In [3]:
conv_base = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet", input_shape=(150,150,3))

58892288/58889256 [==============================] - 10s 0us/step


In [4]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [7]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count,3))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

test_features, test_labels = extract_features(test_dir, 1500)

Found 1500 images belonging to 3 classes.


In [8]:
validation_features, validation_labels = extract_features(validation_dir, 1500)
train_features, train_labels = extract_features(train_dir, 3000)

Found 1500 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.


In [9]:
test_features_reshaped=np.reshape(test_features, (1500, 4*4*512))
train_features_reshaped = np.reshape(train_features, (3000, 4 * 4 * 512))
validation_features_reshaped = np.reshape(validation_features, (1500, 4 * 4 * 512))

In [10]:
conv_base.trainable = False

In [12]:
# feature extraction model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer, Dropout
from keras import models
from keras import layers
from keras import optimizers

model= models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_features_reshaped, train_labels,
                    epochs=100,
                    batch_size=20,
                    validation_data=(validation_features_reshaped, validation_labels), 
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8),
                    tf.keras.callbacks.ModelCheckpoint('/Users/sandy/Desktop', monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False)]
)

Epoch 1/100
150/150 [==============================] - 2s 8ms/step - loss: 0.6269 - acc: 0.7761 - val_loss: 0.2163 - val_acc: 0.9153
INFO:tensorflow:Assets written to: /Users/sandy\Desktop\assets
Epoch 2/100
150/150 [==============================] - 1s 8ms/step - loss: 0.1966 - acc: 0.9271 - val_loss: 0.1514 - val_acc: 0.9373
INFO:tensorflow:Assets written to: /Users/sandy\Desktop\assets
Epoch 3/100
150/150 [==============================] - 1s 8ms/step - loss: 0.1644 - acc: 0.9412 - val_loss: 0.1528 - val_acc: 0.9407
Epoch 4/100
150/150 [==============================] - 1s 8ms/step - loss: 0.1191 - acc: 0.9579 - val_loss: 0.1448 - val_acc: 0.9440
INFO:tensorflow:Assets written to: /Users/sandy\Desktop\assets
Epoch 5/100
150/150 [==============================] - 1s 7ms/step - loss: 0.1136 - acc: 0.9531 - val_loss: 0.1492 - val_acc: 0.9340
Epoch 6/100
150/150 [==============================] - 1s 8ms/step - loss: 0.1016 - acc: 0.9616 - val_loss: 0.1070 - val_acc: 0.9607
INFO:tensorfl

In [13]:
#using the vgg base
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer, Dropout
from keras import models
from keras import layers
from keras import optimizers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [14]:
conv_base.trainable = False

In [15]:
from keras.preprocessing.image import ImageDataGenerator
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=8)
train_datagen = ImageDataGenerator(
      rescale=1./255,)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        
        class_mode='categorical', shuffle=False)

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical', shuffle=False)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-5),
              metrics=['acc'])

history = model.fit(
      train_generator,
      steps_per_epoch=20,
      epochs=100, validation_data=validation_generator, callbacks=[callback])

Found 3000 images belonging to 3 classes.
Found 1500 images belonging to 3 classes.
Epoch 1/100
20/20 [==============================] - 65s 3s/step - loss: 1.1543 - acc: 0.2689 - val_loss: 0.9761 - val_acc: 0.5307
Epoch 2/100
20/20 [==============================] - 68s 4s/step - loss: 0.9655 - acc: 0.5454 - val_loss: 0.9173 - val_acc: 0.6120
Epoch 3/100
20/20 [==============================] - 67s 3s/step - loss: 0.8840 - acc: 0.6735 - val_loss: 0.8394 - val_acc: 0.6760
Epoch 4/100
20/20 [==============================] - 72s 4s/step - loss: 0.7666 - acc: 0.7296 - val_loss: 0.7856 - val_acc: 0.6987
Epoch 5/100
20/20 [==============================] - 78s 4s/step - loss: 0.8747 - acc: 0.6131 - val_loss: 0.7542 - val_acc: 0.6993
Epoch 6/100
20/20 [==============================] - 77s 4s/step - loss: 0.6849 - acc: 0.7877 - val_loss: 0.7240 - val_acc: 0.7133
Epoch 7/100
20/20 [==============================] - 75s 4s/step - loss: 0.7115 - acc: 0.7639 - val_loss: 0.6742 - val_acc: 0.7700

In [23]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20, shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator, steps=50)
print('test acc:', test_acc)

Found 1500 images belonging to 3 classes.
50/50 [==============================] - 36s 714ms/step - loss: 0.2549 - acc: 0.9180
test acc: 0.9179999828338623


In [24]:
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)

predictions = model.predict(test_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

In [25]:
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())   

In [26]:
from sklearn import metrics
report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)    

              precision    recall  f1-score   support

    lung_aca       0.93      0.85      0.89       500
      lung_n       0.94      0.99      0.96       500
    lung_scc       0.91      0.95      0.93       500

    accuracy                           0.93      1500
   macro avg       0.93      0.93      0.93      1500
weighted avg       0.93      0.93      0.93      1500



In [27]:
predictions = model.predict(test_generator)

In [29]:
Y_pred = model.predict(validation_generator, test_generator.samples / test_generator.batch_size)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

Confusion Matrix
[[414  24  62]
 [  7 493   0]
 [ 22   0 478]]
